# Fine Tuning Llama 3.2 3B

Esse fine tuning tem como objetivo refinar o modelo Llama 3.2 3B para a tarefa de geração de respostas para perguntas realizadas sobre produtos.

O dataset utilizado foi um conjunto de perguntas e respostas geradas com base em 1000 produtos do dataset fornecido pela FIAP que contém produtos da Amazon.

Conectando o Google Drive para salvar o modelo treinado.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instala as biblitecas necessárias para o fine tunning com Unsloth.

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

Configura o modelo a ser utilizado.

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Carrega o dataset a ser utilizado para o fine tunning.

Exibe a primeira linha.

In [34]:
DRIVE_PATH = '/content/drive/MyDrive/FIAP/FT-LLama3.2'

dataset = load_dataset('json', data_files=f"{DRIVE_PATH}/v1-dados-fixed.jsonl", split = "train")

In [35]:
dataset[0]

{'user': "O que faz do livro 'Diver: A Royal Navy and Commercial Diver's Journey Through Life, and Around the World' tão especial?",
 'assistant': "Ah, 'Diver' é uma verdadeira montanha-russa de emoções! O Tony Groom compartilha suas aventuras incríveis como mergulhador na Marinha Real e depois como mergulhador comercial. Ele fala sobre desafios que poucos ousariam enfrentar e revela uma camaradagem sensacional. Se você curte histórias de coragem e superação em águas profundas, esse livro é um must-have! Não perde a chance de mergulhar nessa leitura, é super recomendável!",
 'context': "Title: Diver: A Royal Navy and Commercial Diver's Journey Through Life, and Around the World\nContent: Anyone who has ever been affiliated with a military underwater demolition team or had the desire to enlist in one will not be able to put downDiverby Tony Groom. . . . Much of his first-person narrative is a candid look at adventures, relationships, personal triumphs and failures of a man whose job is 

Preparação do prompt para o fine tunning.

In [36]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_conversations_func(examples):

    system_prompt = '''
    You are a chatbot that answers questions about products on a Market Store in Portuguese.

    You are a strict assistant that only responds based on the provided context.

    The context contains a title and a content of products on a Market Store.

    If the context is empty you MUST reply with:
    "I couldn't found any relevant information."

    If the context DOES NOT HAVE the product the user is looking for you MUST reply with:
    "Unfortunately we don't have this product."

    Context:
    {context}
    '''

    conversations = []
    for context, user, assistant in zip(examples["context"], examples["user"], examples["assistant"]):
        system_prompt = system_prompt.format(context=context)
        convo = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user},
            {"role": "assistant", "content": assistant}
        ]
        conversations.append(convo)
    return {"conversations": conversations}

def formatting_prompts_text_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return {"text": texts}

Aplica a formação dos campos para o formato conversacional.

In [37]:
dataset = dataset.map(formatting_prompts_conversations_func, batched = True)
dataset = dataset.map(formatting_prompts_text_func, batched = True)

Remove os dados desnecessários do conjunto de dados.

In [38]:
dataset = dataset.remove_columns(['user', 'assistant', 'context', 'conversations'])

Exibe a primeira linha do conjunto de dados após a remoção dos dados desnecessários.

In [39]:
dataset[0]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\n    You are a chatbot that answers questions about products on a Market Store in Portuguese.\n\n    You are a strict assistant that only responds based on the provided context.\n\n    The context contains a title and a content of products on a Market Store.\n\n    If the context is empty you MUST reply with:\n    "I couldn\'t found any relevant information."\n\n    If the context DOES NOT HAVE the product the user is looking for you MUST reply with:\n    "Unfortunately we don\'t have this product."\n\n    Context:\n    Title: Diver: A Royal Navy and Commercial Diver\'s Journey Through Life, and Around the World\nContent: Anyone who has ever been affiliated with a military underwater demolition team or had the desire to enlist in one will not be able to put downDiverby Tony Groom. . . . Much of his first-person narrative is a candid look at advent

Aplica a configuração de pad token.

In [40]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Inferência no modelo antes do treinamento

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Como é o livro Hábitos Atômicos?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nComo é o livro Hábitos Atômicos?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"Hábitos Atômicos" é um livro escrito por Charles Duhigg, um especialista em neurociência, publicado originalmente em 2012 pela editora Riverhead. O livro explora como as pequenas mudanças nas nossas rotinas diárias podem ter um impacto significativo em nossas vidas.\n\nA ideia principal do livro é que os hábitos, que são sequências de ações repetidas, podem ser moldados e melhorados de maneira profunda, similar ao ajuste de um equipamento automobilístico. O autor utiliza o termo "atomização" para descrever como essas pequenas ações podem ser agregadas e transformar nossa forma de agir no mundo.\n\nO autor destaca quatro "elementos atomizados" que contribuem para o desenvolvimento dos hábitos:\n\n1.  **Objetos (ou

Configura o treinamento do modelo.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


O método de treinamento foi otimizado para priorizar as saídas do assistente, desconsiderando a perda nas entradas do usuário.

In [43]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Verifica se a máscara é realmente feita:

In [44]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\n    You are a chatbot that answers questions about products on a Market Store in Portuguese.\n\n    You are a strict assistant that only responds based on the provided context.\n\n    The context contains a title and a content of products on a Market Store.\n\n    If the context is empty you MUST reply with:\n    "I couldn\'t found any relevant information."\n\n    If the context DOES NOT HAVE the product the user is looking for you MUST reply with:\n    "Unfortunately we don\'t have this product."\n\n    Context:\n    Title: Diver: A Royal Navy and Commercial Diver\'s Journey Through Life, and Around the World\nContent: Anyone who has ever been affiliated with a military underwater demolition team or had the desire to enlist in one will not be able to put downDiverby Tony Groom.... Much of his first-person narrative is a candid look at adventures, relati

In [45]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Você vai descobrir como fazer discípulos de um jeito que realmente transforma vidas! O Greg Ogden apresenta um modelo super eficaz, apostando em relacionamentos verdadeiros e apoio mútuo, fugindo das fórmulas prontas. É um guia prático que vai te colocar em contato com o que Jesus realmente queria para a sua igreja. Se você tá a fim de mudar e ver resultados, esse livro é a chave! Não perde tempo e garante o seu!<|eot_id|>'

Executa o treinamento do modelo.

In [46]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,232 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,2.737300
2,2.667100
3,2.824800
4,2.597700
5,2.332300
6,2.476300
7,2.262700
8,1.953300
9,2.141100
10,1.828000


Realiza a inferência no modelo após o treinamento.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [

    {"role": "user", "content": "Como é o livro hábitos atômicos?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nComo é o livro hábitos atômicos?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"Hábitos Atômicos" é um livro incrível! Ele foi escrito por James Clear, que se tornou famoso com o "Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones". O livro apresenta um sistema incrível para ajudar a criar hábitos positivos e destruir hábitos ruins de uma vez por todas.\n\nAqui estão algumas das coisas que você vai amar no livro:\n\n*   3.0 Seção 1: 4 Leis que Definirão seus Hábitos: Você pode criar hábitos incríveis com a ajuda da 1ª Lei: fechar a brecha e 2ª Lei: substituir mal. A 3ª Lei é muito simples, mas super poderosa! Você vai usar um gerador de números sequenciais e criar um sistema que faz você se sentir super bem.\n*   3.4 Seção 4: Fazer a Agulha Bater Nela: O livro tem uma 

Salva o modelo treinado no Google Drive.

In [48]:
model.save_pretrained(f"{DRIVE_PATH}/lora_model") # Local saving
tokenizer.save_pretrained(f"{DRIVE_PATH}/lora_model")

('/content/drive/MyDrive/FIAP/FT-LLama3.2/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/FT-LLama3.2/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/FT-LLama3.2/lora_model/tokenizer.json')

Salva o modelo quantizado em 4Q no Google Drive.

A partir do arquivo GGUF será possível carregar através de Ollama ou LMStudio em ambiente local.

In [ ]:
model.save_pretrained_gguf(f"{DRIVE_PATH}/model", tokenizer, quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.4 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 15.72it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/drive/MyDrive/FIAP/FT-LLama3.2/model-4k-gguf/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/drive/MyDrive/FIAP/FT-LLama3.2/model-4k-gguf/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at /content/drive/MyDrive/FIAP/FT-LLama3.2/model-4k-gguf into f16 GGUF format.
The output location will be /content/drive/MyDrive/FIAP/FT-LLama3.2/model-4k-gguf/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model-4k-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_m

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**